"""
## Caso: Bebidas carbonatadas

El problema del transporte de una conocida empresa de bebidas. El cual tiene dos plantas de producción en Perú, ubicadas en Lima y Arequipa, y necesita distribuir sus productos a varios centros de distribución en ciudades como Trujillo, Cusco, Iquitos, Tacna, Piura, Chiclayo y Puno.

La idea es utilizar estos datos para mostrar cómo se puede aplicar la programación lineal para optimizar los costos de transporte en un contexto empresarial.

A continuación, se presenta el código en Python utilizando Pyomo para modelar y resolver este problema de transporte.
"""

In [ ]:
# Importar Biblioteca

In [89]:
from pyomo.environ import *

In [ ]:
# Datos del problema
# Plantas de producción (capacidad en cajas)

In [91]:
Plantas = {'Lima': 1200, 'Arequipa': 1000}

In [ ]:
# Centros de distribución (demanda en cajas)

In [92]:
Centros = {
    'Trujillo': 200,
    'Cusco': 150,
    'Iquitos': 100,
    'Tacna': 150,
    'Piura': 200,
    'Chiclayo': 150,
    'Puno': 100
}

In [ ]:
# Definición de los costos de transporte
# Este diccionario contiene los costos de transporte por caja de bebidas
# desde cada planta de producción hasta cada centro de distribución.
# Los costos están en soles peruanos y son estimaciones basadas en la distancia.
# Cada clave del diccionario es un par (planta, centro de distribución),
# y el valor asociado es el costo por caja.
# Por ejemplo, el costo de enviar una caja desde Lima a Trujillo es de 3.5 soles.

# En la vida real, determinar estos costos de transporte puede ser la parte más complicada
# del problema. Requiere investigación detallada y a menudo negociación con proveedores de
# transporte. Un buen método aproximado, especialmente cuando se dispone de poca información,
# es utilizar la distancia como un factor determinante. Se puede calcular o estimar un costo
# por unidad de transporte por kilómetro, y luego multiplicar este costo por la distancia
# entre cada origen y destino. Esta aproximación proporciona una base razonable para modelar
# los costos de transporte en un escenario como el presente, donde los datos exactos pueden
# no estar disponibles o ser difíciles de obtener.

In [93]:
Costos_transporte = {
    ('Lima', 'Trujillo'): 3.5,
    ('Lima', 'Cusco'): 4.0,
    ('Lima', 'Iquitos'): 5.0,
    ('Lima', 'Tacna'): 4.5,
    ('Lima', 'Piura'): 3.2,
    ('Lima', 'Chiclayo'): 3.4,
    ('Lima', 'Puno'): 4.5,
    ('Arequipa', 'Trujillo'): 4.5,
    ('Arequipa', 'Cusco'): 3.0,
    ('Arequipa', 'Iquitos'): 5.5,
    ('Arequipa', 'Tacna'): 2.5,
    ('Arequipa', 'Piura'): 4.2,
    ('Arequipa', 'Chiclayo'): 4.0,
    ('Arequipa', 'Puno'): 3.2
}

In [ ]:
# Crear el modelo

In [94]:
modelo = ConcreteModel()

In [ ]:
# Definición de las variables de decisión
# 'modelo.transporte' representa la cantidad de cajas de bebidas que se enviarán
# desde cada planta de producción a cada centro de distribución.
# Las claves de esta variable multidimensional corresponden a cada par
# (planta de producción, centro de distribución).
# La 'domain' especificada como 'NonNegativeReals' significa que
# la cantidad de cajas enviadas no puede ser negativa, reflejando una realidad física
# donde no tiene sentido enviar una cantidad negativa de productos.

In [96]:
modelo.transporte = Var(Plantas.keys(), Centros.keys(), domain= NonNegativeReals)

In [ ]:
# Definición de la función objetivo
# 'modelo.costo_total' representa el costo total de transporte de todas las cajas
# desde las plantas hasta los centros de distribución.
# La expresión suma el producto del costo de transporte por caja (definido en
# 'Costos_transporte') y la cantidad de cajas a transportar (definida en
# 'modelo.transporte') para cada combinación de planta y centro de distribución.
# La 'sense' de la función objetivo es 'minimize', lo que indica que el
# objetivo del modelo es minimizar esta suma total de costos de transporte.
# Esto es coherente con la meta de encontrar la forma más económica de
# distribuir las cajas desde las plantas a los centros de distribución.

In [97]:
modelo.costo_total = Objective(
    expr= sum(Costos_transporte[planta,centro] * modelo.transporte[planta,centro]
             for planta in Plantas for centro in Centros), sense = minimize)

In [ ]:
# Definición de las restricciones de oferta para las plantas
# 'modelo.restriccion_oferta' es una lista de restricciones que asegura que
# la cantidad total de cajas enviadas desde cada planta no exceda su capacidad de producción.
# Para cada planta, sumamos la cantidad de cajas enviadas a todos los centros de distribución.
# Esta suma debe ser menor o igual a la capacidad de producción de la planta,
# que está definida en el diccionario 'Plantas'.
# Estas restricciones son cruciales para garantizar que el modelo no planifique
# enviar más productos de los que cada planta puede producir.

In [98]:
modelo.restriccion_oferta = ConstraintList()
for planta in Plantas:
    modelo.restriccion_oferta.add(
        sum(modelo.transporte[planta, centro] for centro in Centros) <= Plantas[planta])

In [ ]:
# Definición de las restricciones de demanda para los centros de distribución
# 'modelo.restriccion_demanda' es una lista de restricciones que asegura que
# la demanda de cada centro de distribución sea completamente satisfecha.
# Para cada centro de distribución, sumamos la cantidad de cajas enviadas desde
# todas las plantas. Esta suma debe ser igual a la demanda del centro de distribución,
# la cual está definida en el diccionario 'Centros'.
# Estas restricciones son fundamentales para garantizar que el modelo planifique
# el envío de suficientes productos para cumplir con la demanda de cada centro.

In [99]:
modelo.restriccion_demanda = ConstraintList()
for centro in Centros:
    modelo.restriccion_demanda.add(
        sum(modelo.transporte[planta, centro] for planta in Plantas) == Centros[centro])

In [ ]:
# Resolver el modelo usando el solucionador GLPK

In [100]:
SolverFactory('glpk').solve(modelo)

{'Problem': [{'Name': 'unknown', 'Lower bound': 3495.0, 'Upper bound': 3495.0, 'Number of objectives': 1, 'Number of constraints': 10, 'Number of variables': 15, 'Number of nonzeros': 29, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.12994861602783203}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [ ]:
# Imprimir los resultados del modelo
# Este bloque de código recorre todas las combinaciones de plantas y centros de distribución.
# Para cada par (planta, centro), obtiene el valor de la variable de decisión 'modelo.transporte',
# que indica la cantidad de cajas a enviar desde esa planta a ese centro.
# Solo imprimimos las cantidades que son mayores que cero, lo que significa que hay un envío real
# desde la planta al centro de distribución.
# Esto ayuda a visualizar de manera clara y concisa qué rutas de transporte se utilizarán
# y cuántas cajas se enviarán por cada ruta, según la solución óptima encontrada por el modelo.

In [102]:
for planta in Plantas:
    for centro in Centros:
        cantidad = modelo.transporte[planta, centro].value
        if cantidad > 0:
            print(f"Desde {planta} enviar a {centro}: {cantidad} cajas")

Desde Lima enviar a Trujillo: 200.0 cajas
Desde Lima enviar a Iquitos: 100.0 cajas
Desde Lima enviar a Piura: 200.0 cajas
Desde Lima enviar a Chiclayo: 150.0 cajas
Desde Arequipa enviar a Cusco: 150.0 cajas
Desde Arequipa enviar a Tacna: 150.0 cajas
Desde Arequipa enviar a Puno: 100.0 cajas
